# 추천 시스템

## 고전적 추천시스템

* 협업 필터링


대규모의 기존 **사용자 행동정보**를 분석하여 해당 사용자와 비슷한성향의 사용자들이 기존에 좋아했던 항목을 추천<br>
결과가 직관적, 기존자료 활용 필요(새로운 자료추가되면 추천 곤란문제-콜드스타트)<br>
계산량이 많음 - 사용자 수 많은 경우 효율적 추천 잘 안됨<br>
롱테일(long-tail)문제 - 사용자는 소수의 인기항목에만 관심 --비대칭적쏠림


**추천시스템 관리항목 많은경우, 협업필터링 한계

* 콘텐츠 기반 필터링


**항목자체**를 분석하여 추천을 구현(ex>음악자체를 분석후 유사음악추천)<br>
item profile - 항목분석<br>
user profile - 사용자선호도 추출 두항목의 유사성 계산<br>
콘텐츠 내용자체 분석 >>아이템분석알고리즘이 핵심(군집분석 - Clustering analysis, 인공신경망, tf-id(term frequency-inverse document frequency)등 사용)

**콜드스타트 문제해결, 다양한형식 항목 추천 어려움(각각항목에서 얻는정보 다르므로)<br>
ex)음악, 사진, 비디오 동시추천 힘드

## 모델기반 협력 필터링

기존 항목간 유사성비교에서 벗어나 자료안에 내재한 패턴을 이용하는 기법


-자료의 크기를 동적으로 변화<br>
-사용자가 특정항목을 선호하는 이유를 알고리즘적으로 알아내는 기법(잠재모델기반)

세부정보유추 >> 높은정확도로 항목추천가능 but 모델만드는데 많은계산필요(LDA, 베이지안네트웍)<br>
최근은 딥러닝기반기술 사용

## 추천 알고리즘과 필터버블

 추천 시스템은 정보를 추려서(filtering) 사용자에게 제공한다.
다시 말하면 사용자가 전체 정보를 볼 기회를 박탈당할 수도 있다는 말이다. 추천 시스템이 고도
화될수록 사용자의 입맛에 맞는 정보만 제공되고 나머지 정보는 감추어지는 위험이 생기는데, 이
러한 현상을 필터버블(filter bubble)이라고 한다

# 데이터 탐색과 전처리

In [1]:
import pandas as pd
import os

In [2]:
fname = os.getenv('HOME')+'/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id','artist_MBID','artist','play'] # 임의로 칼럼지정
data = pd.read_csv(fname, sep='\t',names=col_names) # sep='\t'해야 tsv열수 있음!!
data.head(10)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# 사용하는 칼럼만 남김 artistid >>artist
using_cols = ['user_id','artist','play']
data = data[using_cols]
data.head(10)

In [ ]:
type(data.loc[0]['artist'])

In [ ]:
data.loc[0,'artist']

In [ ]:
# 검색쉽게하기위해 아티스트를 소문자로
data['artist'] = data['artist'].str.lower()
data.head(10)

In [ ]:
# 첫번째유저가 듣는 노래?
condition = data['user_id']==data.loc[0,'user_id']

In [ ]:
data[condition] #data.loc[condition]

## 데이터 탐색

기본정보<br>
* 유저수, 아티스트수, 인기많은 아티스트<br>
* 유저들이 몇명의 아티스트를 듣고있는지 통계<br>
* 유저 play횟수 중앙값에 대한 통계

In [ ]:
# 유저수
data['user_id'].nunique()

In [ ]:
len(set(data['user_id']))

In [ ]:
# 아티스트 수
data['artist'].nunique()

In [ ]:
artist_sum = data.groupby('artist')['play'].sum()

In [ ]:
artist_sum.head()

In [ ]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()

In [ ]:
artist_count.sort_values(ascending=False).head(10)

In [ ]:
# 유저별 몇명의 아티스트를 듣고 있는지에 대한 통계
# groupby뒤에 함수까지 붙어야 series로 나옴
user_count = data.groupby('user_id')['artist'].count()

In [ ]:
type(user_count)

In [ ]:
user_count.describe()

In [ ]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

## 모델검증을 위한 사용자 초기정보 세팅

* 나는 이걸 좋아한다.

In [ ]:
# 본인이 좋아하는 아티스트 데이터 추가
my_favorite = ['black eyed peas', 'maroon5', 'jason mraz', 'coldplay', 'beyoncé']

# 'zimin'이라는 userid가 위 아티스트의 노래를 30회씩 들었다 가정
my_playlist = pd.DataFrame({'user_id':['zimin']*5, 'artist':my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any(): #any 적어도 한번
    data = data.append(my_playlist)

data.tail(10)

In [ ]:
data[data.isin({'user_id':['zimin']})['user_id']]

## 모델에 활용하기 위한 전처리(실습)

* user, artist에 번호부여 -  인덱싱

In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 인덱싱하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)} #인덱스번호와 리스트의 값
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [ ]:
# 인덱싱 결과 확인
print(user_to_idx['zimin']) # 마지막유저니 마지막번호가 떠야함
print(artist_to_idx['black eyed peas'])

In [ ]:
# 인덱싱을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dict자료형의 get함수 https://wikidocs.net/16

# user_to_idx.get을 통해 user_id 컬럼의 모든값을 인덱싱한 series 구함
# 정상적으로 인덱싱 안된 row는 nan이 될테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
# map(int, input().split())
# data['user_id'] iterable feature(list, seq)
# map(~~) function
if len(temp_user_data)==len(data): # 모든 row가 정상 인덱싱이면
    print('user_id_column indexing OK!')
    data['user_id']=temp_user_data #data['user_id']를 인덱싱된 series로 교체
else:
    print('user_id column indexing Fail!')
    
# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

In [ ]:
data

In [ ]:
data.tail(10)

## 사용자의 명시적/암묵적 평가

* 명시적평가 : 좋아요, 별점등 선호도를 명시적으로 나타냄
    

* 암묵적 평가 : 구매횟수, 재생횟수등 서비스 사용하며 발생하는 피드백

In [ ]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}') #.2f or .2% 소수점 둘째까지 표시
# f-format에 대한 설명은 https://bit.ly/2DTLqYU

한 번이라도 들었으면 선호한다고 판단한다.<br>
많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

# Matrix Factorization (MF) ??

# CSR(Compressed Sparse Row)Matrix

* CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

In [ ]:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]]
# = data[k]., M,N은 matrix의 shape

In [ ]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play,(data.user_id, data.artist)), shape=(num_user, num_artist))
csr_data

## MF 모델 학습하기

* MF모델을 implict패키지를 사용하여 학습

implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* AlternatingLeastSquares parameter

-1. factors : 유저와 아이템의 벡터 차원수<br>
-2. regulariization : 과적합 방지위해 정규화값 얼마나 사용<br>
-3. use_gpu : GPU사용할지<br>
-4. iterarions : epochs(몇번 반복학습)

In [ ]:
#implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als모델은 input으로(item X user꼴의 matrix를 받기 때문에 Transpose해줌)
csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
# 모델훈련
als_model.fit(csr_data_transpose)

In [ ]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

In [ ]:
zimin_vector

In [ ]:
black_eyed_peas_vector

In [ ]:
# zimin과 black_eyed_peas를 내적하는 코드
# np.dot 행렬곱
np.dot(zimin_vector, black_eyed_peas_vector)

In [ ]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

## 비슷한 아티스트 찾기+유저에게 추천하기

In [ ]:
#simililar_items메서드 이용
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

In [ ]:
# (id, 유사도)Tuple의 id를 다시 이름으로 매핑
# artist_to_idx를 뒤집어, index로부터 artist이름을 얻는 dict생성
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

In [ ]:
# 함수화
def get_similar_artist(artist_name:str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print('슝=3')

In [ ]:
get_similar_artist('2pac')

In [ ]:
get_similar_artist('lady gaga')

## 유저에게 아티스트 추천하기

In [ ]:
# recommend 메소드 통해 자신이 좋아할만한 아티스트 추천받기
# filter_already_liked_items : 이미 평가한 아이템은 제외
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받는다
artist_recommend = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommend

In [ ]:
[idx_to_artist[i[0]] for i in artist_recommend]

In [ ]:
# explain메소드 : 기록을 남긴 데이터중 이 추천에 기여한 정도 확인 가능
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [ ]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]